# Utils

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path
from datetime import datetime, timedelta
from scipy.stats import skew, skewnorm
import random
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
#| export
def get_season(date):
  month = date.month
  if month  >= 3 and month <= 5:
    return "Autumn"
  elif month >= 6 and month <= 8:
    return "Winter"
  elif month >= 9 and month <= 11:
    return "Spring"
  else:
    return "Summer"

In [1]:
#| export
def series_timestamps(ser):
    result = []
    start, end = None , None
    # Loop through the input list
    for row in ser.items():
        # If we encounter a 1, update the start and end index if needed
        if row[1] == 1:
            if start is None:
                start = row[0]
            end = row[0]
        # If we encounter a 0, append the start and end index to the result
        # and reset the start and end index
        elif row[1] == 0 and start is not None:
            result.append((start, end))
            start = None
            end = None

    # If the last element of the input list is a 1, we need to append
    # the start and end index to the result one more time
    if start is not None:
        result.append((start, end))
    return result

Bad pipe message: %s [b"\x9b\xb4\xe7\xf53$\xb6k\xbe\x88\xc9\x97\x1b\xd4\x9c\x82\x91\xd5\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08", b'\x08\n\x08\x0b\x08\x04\x08', b'\x06\x04\x01\x05']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'', b'\x03\x03']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'', b'\x02']
Bad pipe message: %s [b'/\t\x1c\xc6gC\\\xc3\xbeMD\xea\xafX\xddkq\xa

In [ ]:
#| export
def plot_sim(strategy, df):
    fig, ax = plt.subplots(nrows=4,figsize=(18,6), sharey="row", gridspec_kw=dict(height_ratios=[.2,.2,.2,3], hspace=0))
    i=0
    # Plot flow data
    im2 = ax[i].imshow(df.energy.values[np.newaxis,:], cmap="viridis", aspect="auto", vmin=0, vmax=18)
    ax[i].set_yticks([])
    ax[i].set_xticks([])

    ax[i].set_ylabel('Energy', rotation=0, fontsize=8)
    ax[i].yaxis.set_label_coords(-0.03,.2)
    i=1
    # Plot flow data
    ax[i].imshow(df.flow.values[np.newaxis,:], cmap="Reds", aspect="auto")
    ax[i].set_yticks([])
    ax[i].set_xticks([])

    ax[i].set_ylabel('Flow', rotation=0, fontsize=8)
    ax[i].yaxis.set_label_coords(-0.03,.2)
    
    i=2
     # Plot flow data
    im = ax[i].imshow(df.cost.values[np.newaxis,:], cmap="jet", aspect="auto")
    ax[i].set_yticks([])
    ax[i].set_xticks([])
    ax[i].set_ylabel('Cost', rotation=0, fontsize=8)
    ax[i].yaxis.set_label_coords(-0.03,.2)



    # add title
    title = f'Simulation:'+ strategy['name'] + '\n' +\
            'mode : ' + strategy['operation'] + ' | ' +\
            'bedrooms : ' + str(strategy['bedrooms']) + '\n' +\
            'element : ' + str(strategy['element']) + 'kW | ' +\
            'ripple : ' + str(strategy['ripple'])
    i=3
    # ax.plot(df.action*1)
    for j in range(2):
        ax[i].plot(df[f'T{j}'], label=f'T{j}')
    ax[i].set_ylim(50,83)
    # ax[0].set_title('Temperature')
    ax[i].set_ylabel('°C', rotation=90)
    ax[i].margins(x=0, y=0)
    ax[i].axhline(y=55, color='k', linestyle='--', alpha = .5)

    # if strategy['Tset_H'] != strategy['Tset_L']:
    #     ax[i].axhspan(strategy['Tset_H']-2, strategy['Tset_H'],0,.1,  color='mistyrose', alpha=0.4) # wants power but ripple control is on
    #     ax[i].annotate('Upper Thermostat', xy=(0, 0.75), xycoords='axes fraction',  xytext=(5, 0), textcoords='offset points', ha="left", va="top", fontsize=8)


    # ax[i].axhspan(strategy['Tset_L']-2, strategy['Tset_L'],0,.1,  color='powderblue', alpha=0.4) # wants power but ripple control is on

    for k in series_timestamps((df.thermostat_base == 1)&(df.ripple_control)&(strategy['ripple'])): ax[i].axvspan(k[0], k[1], 0.08, .13, color='red', alpha=0.5, hatch ='\\', ec='black') # wants power but ripple control is on
    for k in series_timestamps((df.thermostat_base == 1)&(df.ripple_control)&(~strategy['ripple'])): ax[i].axvspan(k[0], k[1], 0.08, .13, color='blue', alpha=0.5) # wants power but ripple control is on
    for k in series_timestamps((df.thermostat_base == 1)&(df.ripple_control)&(~strategy['ripple'])): ax[i].axvspan(k[0], k[1], 0.08, .13, color='blue', alpha=0.5) # wants power but ripple control is on
    for k in series_timestamps((df.thermostat_base == 1)&(~df.ripple_control)): ax[i].axvspan(k[0], k[1], 0.08, .13, color='green', alpha=0.5)
    for k in series_timestamps((df.action == 1)&(df.thermostat_high == 1)&(~strategy['ripple'])&(df.thermostat_base == 0)): ax[i].axvspan(k[0], k[1],  0.08, .13,color='cyan', alpha=0.5)
    
    
    operation_legend = [Patch(facecolor='red', hatch ='\\', ec='black', alpha=0.5 , label='Demand / No Power'),
                      Patch(facecolor='blue', alpha=0.5 ,  label='Operation during Ripple Control'),
                      Patch(facecolor='cyan', alpha=0.5 ,  label='Boost Operation'),
                      Patch(facecolor='green', alpha=0.5 ,   label='Base Operation')]

    ax[i].axhline(y=55, color='k', linestyle='--', alpha = .5)

    for k in series_timestamps((df.ripple_control)): ax[i].axvspan(k[0], k[1], 0, .05, color='red', alpha=0.5) # wants power but ripple control is on
    for k in series_timestamps((df.peak)): ax[i].axvspan(k[0], k[1], 0, .05, fc='none',hatch ='xx', ec='grey') # wants power but ripple control is on
    for k in series_timestamps((~df.ripple_control)): ax[i].axvspan(k[0], k[1], 0, .05, color='green', alpha=0.5) # wants power but ripple control is on
    network_legend = [Patch(facecolor='red', alpha=0.5 , label='Ripple Control'),
                      Patch(facecolor='green', alpha=0.5 ,  label='Normal Operation'),
                      Patch(facecolor='none', hatch='xx', ec='grey', label='Peak Demand')]


    legend1 = plt.legend(handles = network_legend,loc='upper center',title="Electricity Network", bbox_to_anchor=(0.2, -0.12), fancybox=True, shadow=False, ncol=3)
    legend2 = plt.legend(handles = operation_legend,loc='upper center',title="Operation", bbox_to_anchor=(0.7,-.12), fancybox=True, shadow=False, ncol=4)

    ax[i].add_artist(legend1)
    ax[i].add_artist(legend2)

    # add a colorbar for the price to the right of the plot
    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.92, 0.1, 0.02, 0.8])
    fig.colorbar(im, cax=cbar_ax)
    cbar_ax.set_ylabel('Price [$/kWh]', rotation=90, labelpad=5, fontsize=6)
    cbar_ax.yaxis.set_label_position('right')
    cbar_ax.yaxis.set_ticks_position('left')
    cbar_ax.yaxis.set_tick_params(pad=5)
    # cbar_ax.set_ylim(0, 0.2)
    cbar1_ax = fig.add_axes([0.97, 0.1, 0.02, 0.8])
    fig.colorbar(im2, cax=cbar1_ax)
    cbar1_ax.set_ylabel('kWh', rotation=90, labelpad=5, fontsize=6)
    cbar1_ax.yaxis.set_label_position('right')
    cbar1_ax.yaxis.set_ticks_position('left')
    cbar1_ax.yaxis.set_tick_params(pad=5)
    
    date_form = DateFormatter('%d %b\n %H:%M')
    ax[-1].xaxis.set_major_formatter(date_form)
    cbar_ax.yaxis.set_tick_params(labelsize=6)
    cbar1_ax.yaxis.set_tick_params(labelsize=6)
    ax[i].annotate('network', xy=(0, 0), xycoords='axes fraction',  xytext=(5, 10), textcoords='offset points', ha="left", va="top", fontsize=8)
    ax[i].annotate('operation', xy=(0, 0), xycoords='axes fraction',  xytext=(5, 28), textcoords='offset points', ha="left", va="top", fontsize=8)
    # add some whitespace to the bottom of the plot
    fig.subplots_adjust(bottom=0.25)
    fig.suptitle(title)
    return 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()